In [1]:
from dataloader import DataLoader

In [2]:
loader = DataLoader()

In [3]:
loader.load_sells_data_from_file('sprzedaz.csv')
df_filtered = loader = loader.prepare_sells_data('0001')

In [4]:
print(df_filtered)

    97   96       95   94       93   92   91       90   89       88  ...   10  \
0  0.0  0.0  1682.54  0.0  4698.09  0.0  0.0  4651.55  0.0  2091.89  ...  0.0   

     9        8        7    6        5      4        3    2        1  
0  0.0  2364.89  2397.36  0.0  3494.97  40.25  1149.73  0.0  4760.31  

[1 rows x 97 columns]


In [5]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


In [6]:
X = df_filtered.iloc[:, :-1]
y = df_filtered.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred))

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.